In [85]:
import matplotlib.pyplot as plt
import math
import numpy as np
import random 

tabe e sigmoid va moshtaghe an

In [86]:
def f(x):
  return 1/(1 + math.exp(-x))

In [87]:
def fprime(x):
  return math.exp(-x)/((1 + math.exp(-x)) ** 2)

neural network ra initialize mikonim ba araye i az tedad neuron ha dar laye haye miani. 

dar har laye 1 node ra be bias ekhtesas midahim

w -> vazn

dw -> taghire vazn

val -> meghdare f har node

inp -> meghdare voroodie node

In [162]:
def init(hidden):
  layout = [3] + hidden + [2]
  
  w = []
  for i in range(1, len(layout)):
    x = layout[i - 1]
    y = layout[i]
    layer = [[0] * y for _ in range(x)]
    for j in range(x):
      for k in range(1, y):
        layer[j][k] = random.random()
    w.append(layer)
  w.append([0])

  dw = []
  for i in range(1, len(layout)):
    x = layout[i - 1]
    y = layout[i]
    layer = [[0] * y for _ in range(x)]
    dw.append(layer)
  dw.append([0])

  val = []
  for i in range(len(layout)):
    layer = [0] * layout[i]
    if(i < len(layout) - 1):
      layer[0] = 1
    val.append(layer)

  inp = []
  for i in range(len(layout)):
    layer = [0] * layout[i]
    inp.append(layer)

  delta = []
  for i in range(len(layout)):
    layer = [0] * layout[i]
    delta.append(layer)

  return w, dw, inp, val, delta, layout

forward propagation ke meghdare har node va input ash ra bar migardanad

In [164]:
def forward(x, y, w, inp, val, layout):
  val[0][1] = x
  val[0][2] = y
  for i in range(1, len(layout)):
    for j in range(1, layout[i]):
      tmp = 0
      for k in range(layout[i - 1]):
        tmp += w[i - 1][k][j] * val[i - 1][k]

      inp[i][j] = tmp
      val[i][j] = f(tmp)

  return val, inp


backward propagation ke taghirate vazn ra barmigardanad

In [166]:
def backward(ans, w, dw, inp, val, delta, layout, alpha):
  
  for i in range(1, len(layout)):
    for j in range(1, layout[-i]):
      tmp = 0
      if(i == 1):
        tmp = ans - val[-1][1]
      else:
        for k in range(1, layout[-i + 1]):
          #print(i, j, k, len(w[-i][j]))
          tmp += delta[-i + 1][k] * w[-i][j][k]
        
      delta[-i][j] = tmp * fprime(inp[-i][j])

      for k in range(layout[-i - 1]):
        dw[-i - 1][k][j] = alpha * delta[-i][j] * val[-i - 1][k]


  return dw

update arayeye w ra update mikonad va barmigardanad

In [168]:
def update(w, dw, layout):
  for i in range(len(layout) - 1):
    for j in range(layout[i]):
      for k in range(1, layout[i + 1]):
        w[i][j][k] += dw[i][j][k]
  return w

cycle yek bar kolle data ra rooye neural network run mikonad va taghirat ra anjam midahad

In [170]:
def cycle(data, alpha, w, dw, inp, val, delta, layout):
  for line in data:
    #print(line)
    x, y, ans = line
    val, inp = forward(x, y, w, inp, val, layout)
    dw = backward(ans, w, dw, inp, val, delta, layout, alpha)
    w = update(w, dw, layout)

  return w, dw, inp, val, delta

train dadeye train ra migirad va neural network ra train mikonad

In [171]:
def train(data, iteration_count, hidden, alpha):
  w, dw, inp, val, delta, layout = init(hidden)

  for _ in range(iteration_count):
    w, dw, inp, val, delta = cycle(data, alpha, w, dw, inp, val, delta, layout)

  return w, dw, inp, val, delta, layout

check dadeye test ra migirad va darsade ghalat ha ra barmigardanad

In [196]:
def check(data, w, inp, val, layout):
  wrong = 0
  for line in data:
    x, y, ans = line
    val, inp = forward(x, y, w, inp, val, layout)
    pred = val[-1][1]
    #print(pred)
    guess = 0
    if(pred >= 0.5):
      guess = 1
    if(guess != ans):
      wrong += 1

  return wrong, wrong / len(data) * 100


In [173]:
fl = open("/content/train", mode = "r")
lines = fl.readlines()
train_data = []
for line in lines:
  nl = list(map(float, line.split()))
  train_data.append(nl)
fl.close()

minx = 10
maxx = -10
miny = 10
maxy = -10
for line in train_data:
  x, y, z = line
  minx = min(x, minx)
  maxx = max(x, maxx)
  miny = min(y, miny)
  maxy = max(y, maxy)

for i in range(len(train_data)):
  train_data[i][0] = (train_data[i][0] - minx) / (maxx - minx)
  train_data[i][1] = (train_data[i][1] - miny) / (maxy - miny)

fl = open("/content/test", mode = "r")
lines = fl.readlines()
test_data = []
for line in lines:
  nl = list(map(float, line.split()))
  test_data.append(nl)

for i in range(len(test_data)):
  test_data[i][0] = (test_data[i][0] - minx) / (maxx - minx)
  test_data[i][1] = (test_data[i][1] - miny) / (maxy - miny)
fl.close()

In [194]:
w, dw, inp, val, delta, layout = train(train_data, 700, [5, 5, 5, 5], 0.2)

In [219]:
w, dw, inp, val, delta, layout = init([15, 15])

behtarin darsade khata (kamtarin = 6.2)

In [220]:
counter = 0
while(True):
  w, dw, inp, val, delta = cycle(train_data, 0.3, w, dw, inp, val, delta, layout)
  
  print(check(test_data, w, inp, val, layout))

  counter = (counter + 1) % 10

(497, 49.7)
(497, 49.7)
(497, 49.7)
(496, 49.6)
(91, 9.1)
(80, 8.0)
(77, 7.7)
(76, 7.6)
(73, 7.3)
(67, 6.7)
(64, 6.4)
(64, 6.4)
(65, 6.5)
(64, 6.4)
(64, 6.4)
(65, 6.5)
(65, 6.5)
(65, 6.5)
(66, 6.6000000000000005)
(66, 6.6000000000000005)
(66, 6.6000000000000005)
(66, 6.6000000000000005)
(67, 6.7)
(67, 6.7)
(67, 6.7)
(67, 6.7)
(67, 6.7)
(67, 6.7)
(67, 6.7)
(67, 6.7)
(67, 6.7)
(67, 6.7)
(67, 6.7)
(67, 6.7)
(67, 6.7)
(67, 6.7)
(67, 6.7)
(67, 6.7)
(67, 6.7)
(67, 6.7)
(67, 6.7)
(67, 6.7)
(67, 6.7)
(67, 6.7)
(67, 6.7)
(67, 6.7)
(67, 6.7)
(66, 6.6000000000000005)
(66, 6.6000000000000005)
(66, 6.6000000000000005)
(66, 6.6000000000000005)
(66, 6.6000000000000005)
(65, 6.5)
(65, 6.5)
(65, 6.5)
(65, 6.5)
(64, 6.4)
(64, 6.4)
(64, 6.4)
(64, 6.4)
(66, 6.6000000000000005)
(65, 6.5)
(65, 6.5)
(66, 6.6000000000000005)
(66, 6.6000000000000005)
(66, 6.6000000000000005)
(66, 6.6000000000000005)
(66, 6.6000000000000005)
(66, 6.6000000000000005)
(66, 6.6000000000000005)
(65, 6.5)
(64, 6.4)
(64, 6.4)
(64, 6.

KeyboardInterrupt: ignored

In [198]:
print(check(test_data, w, inp, val, layout))

(64, 6.4)
